**Code used in the article:**

_Life Cycle Assessment of pilot scale production of seaweed-based plastic_	

Maddalen Ayala ^1*, Marianne Thomsen^2, Massimo Pizzol^1			
			
^1 Danish Center for Environmental Assessment, Department of Planning, Aalborg University, Rendsburggade 14, 9000 Aalborg, Denmark			
^2 Department of Food Science, University of Copenhagen, Rolighedsvej 26, 1958 Frederiksberg, Denmark 			
			
*Corresponding author e-mail: mace@plan.aau.dk

# Initial setup

In [ ]:
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lci_to_bw2 import lci_to_bw2

In [ ]:
bw.projects.set_current('PlastiSea2')
bw.databases

In [ ]:
# Import the biosphere3 database
#bw.bw2setup()

In [ ]:
# Import ecoinvent

# You need to change the line below with the directory where you have saved ecoinvent
#ei38dir = "/Users/massimo/Documents/Databases/ecoinvent v3.8.1/ecoinvent 3.8_consequential_ecoSpold02/datasets"

#if 'ecoinvent 3.8_conseq' in bw.databases:
#    print("Database has already been imported")
#else:
#    ei38 = bw.SingleOutputEcospold2Importer(ei38dir, 'ecoinvent 3.8_conseq') # You can give it another name of course
#    ei38.apply_strategies()
#    ei38.statistics()

#ei38.drop_unlinked(True)
#ei38.write_database() # This will take some time.

# Import excel data of foreground system

In [ ]:
# create .csv files with inventories
inventories = pd.read_excel('../Data/Seaweed-plastic-inventory.xlsx', sheet_name=None, index_col = 0, header = 0)
for key in inventories.keys(): 
    inventories[key].to_csv('../Data/%s.csv' %key)

In [ ]:
# list of foreground database names for this project
myfgdatabases = [i for i in inventories.keys()]
myfgdatabases

In [ ]:
# Creating brightway dictionaries and writing them as databases
for fgdb in myfgdatabases:
    mydb = pd.read_csv( (fgdb +'.csv'), header = 0, sep = ",")
    
    mydb = mydb.drop(['Simapro name','Notes',
                  'Pedigree Reliability','Pedigree Completeness',
                  'Pedigree Temporal correlation','Pedigree geographical correlation',
                  'Further technological correlation'], 1)  # remove the columns not needed
    
    mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
    
    #Check that units in foreground-bagkround math
    mycodes = mydb.loc[mydb['Exchange database'] == "ecoinvent 3.8_conseq"]['Exchange input']
    myunits = mydb.loc[mydb['Exchange database'] == "ecoinvent 3.8_conseq"]['Exchange unit']
    bwunits = [bw.Database('ecoinvent 3.8_conseq').get(i)['unit'] for i in mycodes]
    checkunits = pd.DataFrame({'codes':mycodes, 'myunits':myunits, 'bwunits': bwunits})
    print(any(checkunits['myunits'] == checkunits['bwunits']))
    
    mydb_dict = lci_to_bw2(mydb) 
    
    if fgdb in bw.databases: 
        del bw.databases[fgdb]
    
    PS_db = bw.Database(fgdb)
    PS_db.write(mydb_dict)


In [ ]:
bw.databases

In [ ]:
# check it worked
activities = [(fgdb, act['code']) for fgdb in myfgdatabases for act in bw.Database(fgdb)]
activities

## Modify the background database to remove all uncertainty

See tutorial here: https://github.com/brightway-lca/brightway2/blob/master/notebooks/Switch%20lognormal%20median%20to%20average.ipynb

**This will take some time!**

In [ ]:
ei_original = bw.Database('ecoinvent 3.8_conseq')

if 'ecoinvent 3.8_conseq no uncertainty' in bw.databases:
    del bw.databases['ecoinvent 3.8_conseq no uncertainty']

db = ei_original.copy('ecoinvent 3.8_conseq no uncertainty')

In [ ]:
data = db.load()

In [ ]:
for key, ds in data.items():
    for exc in ds.get('exchanges', []):
        exc["uncertainty type"] = 0

In [ ]:
db.write(data)
db.process()

In [ ]:
# check keys are the same
original_db_keys = sorted(bw.Database("ecoinvent 3.8_conseq").load().keys())
modified_db_keys = sorted(db.load().keys())
original_db_keys[:3], modified_db_keys[:3]

In [ ]:
# check uncertainty is not there
testexc = list(db.random().exchanges())[0]
print(testexc)
print(testexc['uncertainty type'])
print(testexc.uncertainty)


## Make a copy of the original foreground database and link it to the background database without uncertainty

In [ ]:
# clear up first in case there are already nbu ("no background uncertainty") databases
for nbu_name in [i for i in bw.databases if ' nbu' in i]:
    #print(nbu_name)
    del(bw.databases[nbu_name])

In [ ]:
bw.databases

In [ ]:
myfgdatabases

In [ ]:
for fg in myfgdatabases:
    
    fg_original = bw.Database(fg)
    
    if fg + ' nbu' in bw.databases:         
        del bw.databases[fg]
    fg_nbu = fg_original.copy(fg + ' nbu') # nbu = No Background Uncertainty
        
    fg_data = fg_nbu.load()
    
    for key, ds in fg_data.items():
        for exc in ds.get("exchanges", []):
            if exc["input"][0] == "ecoinvent 3.8_conseq":
                exc["input"] = ("ecoinvent 3.8_conseq no uncertainty", exc["input"][1])
    
    fg_nbu.write(fg_data)
    fg_nbu.process()

In [ ]:
bw.databases

In [ ]:
# Quick LCA

#myact = bw.Database('PS pilot').get('Nutrients')
myact = bw.Database('PS pilot (base)').get('Use and biowaste')
#myact = bw.Database('PS pilot (base) nbu').get('Treatment, biowaste')
#myact = bw.Database("ecoinvent 3.8_conseq").get('8bd2cd7f7a485472795f30c522636466')
print(myact)
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mymethod
functional_unit = {myact : 1}
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

In [ ]:
# Quick comparative MC (independent sampling, likely overestimating variance)
samples = 250 
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')

# first with bg uncertainty 
myact = bw.Database('PS pilot (base)').get('Use and biowaste')
mc = bw.MonteCarloLCA({myact : 1}, mymethod)
mc_results = [next(mc) for x in range(samples)] 
print(np.median(mc_results),np.std(mc_results))

# now without bg uncertainty
_myact = bw.Database('PS pilot (base) nbu').get('Use and biowaste')
_mc = bw.MonteCarloLCA({_myact : 1}, mymethod) 
_mc_results = [next(_mc) for x in range(samples)] 
print(np.median(_mc_results), np.std(_mc_results))


In [ ]:
# plot
df = pd.DataFrame({'bu': mc_results, 'nbu': _mc_results})
df.plot(kind = 'box')
plt.xticks(rotation=90)

In [ ]:
# more plot 
plt.hist(mc_results)
plt.hist(_mc_results)

In [ ]:
# stats
df.describe()

# Create a LCIA method for biogenic Carbon Dioxide

In [ ]:
ipcc13_bio_cf = [[('biosphere3', '73ed05cc-9727-4abf-9516-4b5c0fe54a16'), 1.0], # 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))
              [('biosphere3', 'baf58fc9-573c-419c-8c16-831ac03203b9'), 30.5]] # 'Methane, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))


ipcc13_bio_key = ('IPCC 2013', 'climate change-biogenic', 'GWP 100a')
ipcc13_bio = bw.Method(ipcc13_bio_key)
ipcc13_bio.validate(ipcc13_bio_cf)
ipcc13_bio.register() 
ipcc13_bio.write(ipcc13_bio_cf)
ipcc13_bio.load()

In [ ]:
# Quick LCA
myact = bw.Database('PS pilot (base)').get('Use and biowaste')
print(myact)

mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
fu = {myact : 1}
lca = bw.LCA(fu, mymethod)
lca.lci()
lca.lcia()
GWfossil = lca.score
print('fossil',lca.score)

lca.switch_method(('IPCC 2013', 'climate change-biogenic', 'GWP 100a'))
lca.lcia()
GWbiogen = lca.score
print('biogenic',lca.score)
print('total',GWfossil+GWbiogen)

# Calculate multiple static results

In [ ]:
# initialize a database aggregating all databases first, to obtain a large tech matrix.

In [ ]:
dummy_db = bw.Database('dummydb')

In [ ]:
dummy_db.write({('dummydb', 'dummyactivity'):{
    'name': 'a dummy activity to initilize multiple databases',# Note that a tuple is used to identify an activity univocally
    'exchanges': [{'input': ('dummydb', 'dummyactivity'), 'amount': 1, 'type': 'production'},
                  {'input': ('PS pilot', 'Seaweed, offshore farm'), 'amount': 0, 'unit' : 'kilogram','type': 'technosphere'},
                  {'input': ('PS pilot (base)', 'Use and biowaste'),'amount': 0,'unit' : 'kilogram','type': 'technosphere'},
                  {'input': ('PS pilot (cellulose)', 'Use and biowaste'),'amount': 0,'unit' : 'kilogram','type': 'technosphere'},
                  {'input': ('PS pilot (mannitol)', 'Use and biowaste'),'amount': 0,'unit' : 'kilogram','type': 'technosphere'},
                  {'input': ('PS pilot (PLA5)', 'Use and biowaste'),'amount': 0,'unit' : 'kilogram','type': 'technosphere'},
                  {'input': ('PS pilot (PLA30)', 'Use and biowaste'),'amount': 0,'unit' : 'kilogram','type': 'technosphere'}]}})

In [ ]:
# Quick LCA
myact = bw.Database('dummydb').get('dummyactivity')
print(myact)
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mymethod
functional_unit = {myact : 1}
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

In [ ]:
myfgdatabases = ['PS pilot', 'PS pilot (base)',
                 'PS pilot (mannitol)', 'PS pilot (cellulose)',
                 'PS pilot (PLA30)','PS pilot (PLA5)']

In [ ]:
alternatives = [(fgdb, act['code']) for fgdb in myfgdatabases for act in bw.Database(fgdb) if 'Use' in act['name']]
alternatives

In [ ]:
results_ccf = []
results_ccb = []

for act in alternatives:
    lca.switch_method(('IPCC 2013', 'climate change', 'GWP 100a'))
    lca.redo_lcia({bw.Database(act[0]).get(act[1]):1})
    results_ccf.append(lca.score)
    lca.switch_method(('IPCC 2013', 'climate change-biogenic', 'GWP 100a'))
    lca.redo_lcia({bw.Database(act[0]).get(act[1]):1})
    results_ccb.append(lca.score)
    

In [ ]:
static_results = pd.DataFrame([results_ccf, results_ccb], index = ['CC fossil', 'CC biogenic'], columns = alternatives).T

In [ ]:
static_results['CC total'] = static_results['CC fossil'] +static_results['CC biogenic']

In [ ]:
static_results

In [ ]:
# export values
static_results.to_csv('../Results/static_results.csv')

# Monte Carlo simulation

### Correction of overestimated MC values.
Some ecoinvent datasets have insanely high uncertainty. This is likely an error or overestimation and skews results so this uncertainty was removed. 
See here: https://stackoverflow.com/questions/72807629/overestimated-monte-carlo-results-in-brightway/72821344#72821344

In [ ]:
myact = bw.Database('ecoinvent 3.8_conseq').get('d31acf4564148f0ef483a140317caf37') #cellulose fibre production' (kilogram, CH, None)>
for exc in list(myact.exchanges()):
    if exc['uncertainty type'] == 5:
        exc['uncertainty type'] = 0
        exc.save()

In [ ]:
# Quick LCA to check
myact = bw.Database('ecoinvent 3.8_conseq').get('d31acf4564148f0ef483a140317caf37') #'tissue paper production, virgin' (kilogram, GLO, None)
#myact = bw.Database('PS pilot (base)').get('Use and biowaste') # can also check using this.

print(myact)
fu = {myact: 1}
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')

lca = bw.LCA(fu, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

mc = bw.MonteCarloLCA(fu, mymethod)  
mc_results = [next(mc) for x in range(50)] 
print(np.median(mc_results))

# Look at the MC results
plt.hist(mc_results, density=True)
plt.ylabel("Probability")
plt.xlabel('lca.score')

### Initialize database

In [ ]:
all_myfgdbs = [i for i in bw.databases if i not in ['biosphere3','ecoinvent 3.8_conseq', 'ecoinvent 3.8_conseq no uncertainty']]
alternatives = [(fgdb, act['code']) for fgdb in all_myfgdbs for act in bw.Database(fgdb) if 'Use' in act['name']]

In [ ]:
alternatives

In [ ]:
prod = {'input': ('dummydb mc', 'dummyactivity'), 'amount': 1, 'type': 'production'}
techn = [{'input': alt, 'amount' : 0, 'type' : 'technosphere', 'unit' : 'kilogram'} for alt in alternatives]
techn.append(prod)


dummy_db_mc = bw.Database('dummydb mc')
dummy_db_mc.write({('dummydb mc', 'dummyactivity'):{
    'name': 'a dummy activity to initilize multiple databases - for montecarlo',# Note that a tuple is used to identify an activity univocally
    'exchanges': techn}})

In [ ]:
dummy_fu = {bw.Database('dummydb mc').get('dummyactivity') : 1}
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mc = bw.MonteCarloLCA(dummy_fu, mymethod) # important to initialize the MC simulation
next(mc)

This will take some time!!!

In [ ]:
dummy_fu = {bw.Database('dummydb mc').get('dummyactivity') : 1}
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mc = bw.MonteCarloLCA(dummy_fu, mymethod) # important to initialize the MC simulation
next(mc)

iterations = 1000
simulations_f = []
simulations_b = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults_b = []
    mcresults_f = []    

    for i in alternatives:
        
        mc.switch_method(('IPCC 2013', 'climate change-biogenic', 'GWP 100a'))
        mc.redo_lcia({bw.Database(i[0]).get(i[1]):1}) 
        mcresults_b.append(mc.score)
        
        mc.switch_method(('IPCC 2013', 'climate change', 'GWP 100a'))
        mc.redo_lcia({bw.Database(i[0]).get(i[1]):1}) 
        mcresults_f.append(mc.score)
    
    simulations_b.append(mcresults_b)
    simulations_f.append(mcresults_f)


In [ ]:
df_f = pd.DataFrame(simulations_f, columns = alternatives)
df_f.head()

In [ ]:
df_b = pd.DataFrame(simulations_b, columns = alternatives)
df_b.head()

In [ ]:
df_t = df_f + df_b

In [ ]:
df_t.describe()

In [ ]:
df_t.plot(kind = 'box')
plt.xticks(rotation=90)

In [ ]:
df_f.to_csv('../Results/mc_CC_fossil.csv')
df_b.to_csv('../Results/mc_CC_biogenic.csv')
df_t.to_csv('../Results/mc_CC_total.csv')
df_t.describe().to_csv('../Results/mc_CC_total_summary_stats.csv')